In [2]:
!pip install pdfplumber

     ---------------------------------------- 56.4/56.4 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 2.9/2.9 MB 16.9 MB/s eta 0:00:00


In [18]:
import re

def extract_customer_details(text):
    Orders_data = []
    lines = text.splitlines()  # Split text by line breaks
    customer_name = None
    Order_id = None
    order_items = []
    customer_address = ""
    addressFlag = True
    customerPinCode = ""
    customerCity = ""
    customerMobileNumber = None
    shipping_charges = None
    total_quantity = None
    total_price = None

    # Define the pattern to match the mobile number
    pattern = r"MOBILE NO\. : (\d{10})"
    # Use regex to find the mobile number
    match = re.search(pattern, text)
    # Extract the matched mobile number
    if match:
        customerMobileNumber = match.group(1)
        
        # Define pattern to extract shipping charges and total quantity
    shipping_charges_pattern = r"SHIPPING CHARGES Rs\.(\d+\.\d+)"
    total_quantity_pattern = r"TOTAL (\d+) Rs\."
    total_price_pattern = r"TOTAL \d+ Rs\.([\d,]+\.\d{2})"
    
    # Extract shipping charges
    shipping_charges_match = re.search(shipping_charges_pattern, text)
    if shipping_charges_match:
        shipping_charges = shipping_charges_match.group(1)
        

    # Extract total quantity
    total_quantity_match = re.search(total_quantity_pattern, text)
    if total_quantity_match:
        total_quantity = total_quantity_match.group(1)
        
        
    # Extract total price
    total_price_match = re.search(total_price_pattern, text)
    if total_price_match:
        total_price = total_price_match.group(1).replace(",", "")  # Remove comma from price
        

    for line_index, line in enumerate(lines):
        if line.startswith("DELIVER To:"):
            # Extract customer name from the next line
            if len(lines) > lines.index(line) + 1:
                customer_name = lines[line_index + 1].strip()
                customer_name = customer_name.split()[0:2]
                customer_name = ' '.join(customer_name)
        elif line.startswith("ORDER # :"):
            # Extract order ID from the current line
            Order_id = line.split(": ")[-1].strip()  # Split on ": " and get the last part
        elif line.startswith("SKU ITEM QTY PRICE"):
            # Start collecting order items
            start_item_index = lines.index(line) + 1
        elif line.startswith("SHIPPING CHARGES"):
            # Stop collecting order items
            order_items = lines[start_item_index:line_index]
        elif line.endswith("10005"):
            # Add first word of the line to address and set flag
            customerCity = line.split()[0]
            customer_address += customerCity + "\n"
        elif "110005, India" in line:
            # Define the pattern to match the desired substring
            statePinCodepattern = r"(.*?), IN\b"

            # Use regex to find the matching substring
            match = re.search(statePinCodepattern, line)

            # Extract the matched substring
            if match:
                customerPinCode = match.group(1).strip()
                customer_address += "".join(customerPinCode) + "\n"
                customerPinCode = customerPinCode.split(",")[1].strip()
        elif line_index >= 2:
            if line.endswith("Opposite Metr-"):
                # If "24, 1st Floor" is found, split and join address (excluding parts after)
                substring_to_remove = "24, 1st Floor, Pusa Road, Opposite Metr-"
                cleaned_address_line = re.sub(re.escape(substring_to_remove), "", line).strip()
                customer_address += "".join(cleaned_address_line) + "\n"
                if len(lines) > lines.index(line) + 1:
                    newline = lines[line_index + 1]
                    sub = "o Pillar No-102, Pusa Road, Pusa Road A-"
                    clean = re.sub(re.escape(sub), "", newline).strip()
                    customer_address += "".join(clean) + "\n"
                addressFlag = False
            elif addressFlag == True:
                # Include line if it doesn't contain "24, 1st Floor" (avoid extra split)
                customer_address += line.strip() + "\n"

#     print(customer_name)
    print(Order_id)
#     print(order_items)
#     print(customer_address)
#     print(customerPinCode)
#     print(customerCity)
#     print(customerMobileNumber)
#     print("Total Price:", total_price)
#     print("Total Quantity:", total_quantity)
#     print("Shipping Charges:", shipping_charges)

    extracted_items = []
    prev=""
    length=len(order_items)
    skus = []
    prices = []
    quantities = []

    for item in order_items:
        # Split the item string on spaces
        if item == 'ok':
            continue
        if len(prev) != 0:
            item = prev + " " + item
            prev = ""
        item_parts = item.split()
        length = len(item_parts)
        if length < 4:
            prev = item
            continue
        print(item)
        # Check if there are enough parts to extract information
        if length >= 4:
            # Extract order ID (first word)
            order_id = item_parts[0]
            if order_id =='Gita' and item_parts[1] =='Rhyme':
                order_id = 'Gita Rhyme Book'
            if order_id == 'Gita_colouring&stickerbo':
                order_id = 'Gita_colouring&stickerbook'
            price = item_parts[length - 1]
            quantity = item_parts[length - 2]

            # Append SKU, price, and quantity to respective lists
            skus.append(order_id)
            prices.append(price)
            quantities.append(quantity)
        else:
            print(f"Warning: Skipping invalid item format: {item}")  # Optional: Log invalid items

    # Create comma-separated strings for SKUs, prices, and quantities
    sku_string = ", ".join(skus)
    price_string = ", ".join(prices)
    quantity_string = ", ".join(quantities)
    Orders_data.append([Order_id,customer_name,
    customer_address,
    customerPinCode,
    customerCity,
    customerMobileNumber,
    sku_string,
    price_string,
    quantity_string,                    
    shipping_charges,
    total_quantity,
    total_price])
    return Orders_data

SyntaxError: invalid syntax (3226939028.py, line 132)

In [19]:
import pdfplumber
from openpyxl import Workbook

def extract_data_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[4]  # Assuming data is on the first page
        mid_x = page.width / 2
        
        # Extract data from the left half of the page
        left_half = page.within_bbox((0, 0, mid_x, page.height))
        left_text = left_half.extract_text()
#         print(left_text)
#         print()
        CustomerData1 = extract_customer_details(left_text)


#         Extract data from the right half of the page
        right_half = page.within_bbox((mid_x, 0, page.width, page.height))
        right_text = right_half.extract_text()
        CustomerData2 = extract_customer_details(right_text)
#         print(right_text)
        # Parse the extracted text to extract customer and order details for each half
        # You might need to use regular expressions or specific patterns to extract the data

# def export_to_excel(data, excel_path):
#     wb = Workbook()
#     ws = wb.active
#     ws.append(["Customer Name", "Customer Address", "Order ID"])
#     ws.append(data)
#     wb.save(excel_path)

def main():
    pdf_path = "shipping.pdf"
    excel_path = "shipping_order_data.xlsx"
    extract_data_from_pdf(pdf_path)
#     export_to_excel(data, excel_path)
    print("Data exported to Excel successfully.")

if __name__ == "__main__":
    main()


404-1069910-5721119
GFK_Eng Gita for Kids Teachings, Sanskrit lessons and Activities Simple an... 1 Rs.120.00
12741
GFK_Hindi Bhagavad Gita Bachcho Ke Liye 1 Rs.122.00
Gita Rhyme Book Gita Rhyme Book 1 Rs.122.00
Gita_Colouring Gita Colouring Sticker Book 1 Rs.122.00
Data exported to Excel successfully.


In [17]:
import pdfplumber
import pandas as pd

def extract_data_from_pdf(pdf_path):
    #make df
    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages):
            # Extract data from the left half of the page
            mid_x = page.width / 2
            left_half = page.within_bbox((0, 0, mid_x, page.height))
            left_text = left_half.extract_text()
    #         print(left_text)
    #         print()
            CustomerData1 = extract_customer_details(left_text)
            print(CustomerData1)

            # Extract data from the right half of the page
            right_half = page.within_bbox((mid_x, 0, page.width, page.height))
            right_text = right_half.extract_text()
            CustomerData2 = extract_customer_details(right_text)
    return df

def main():
    pdf_path = "shipping.pdf"
    df = extract_data_from_pdf(pdf_path)

    # Save DataFrame to Excel
    excel_path = "shipping_order_data.xlsx"
    df.to_excel(excel_path, index=False)
    print("Data exported to Excel successfully.")

if __name__ == "__main__":
    main()


12736
GFBS_English Gita Family Book Set 1 Rs.690.00
[['12736', 'Kiran Shyam', 'STPI Building 4th Floor SDA Complex Kas-\numpti\n\nShimla\nHimachal Pradesh, 171009\n', '171009', 'Shimla', '9418647061', 'GFBS_English', 'Rs.690.00', '1', '80.00', '1', '770.00']]
171-5526488-0880333
GFK_Eng Gita for Kids Teachings, Sanskrit lessons and Activities Simple an... 1 Rs.120.00
408-0891930-8462724
GFK_Eng Gita for Kids Teachings, Sanskrit lessons and Activities Simple an... 1 Rs.120.00
[['408-0891930-8462724', 'Gopika P', 'Flat No 305 Block A Vineyard Chrysolite-\n, Basappa Layout 1st Cross, Hennur Vill-\nage Road Kalyan Nagar P O\nBENGALURU\nKARNATAKA, 560043\n', '560043', 'BENGALURU', '8495064434', 'GFK_Eng', 'Rs.120.00', '1', '30.00', '1', '150.00']]
404-6319842-6876317
GFK_Eng Gita for Kids Teachings, Sanskrit lessons and Activities Simple an... 1 Rs.120.00
12737
GFK_English Gita For Kids 1 Rs.122.00
[['12737', 'Rahul Saxena', 'Sobha Suncrest Thalagattapura Main Road\nNagegowdanapalya 1083, 1

NameError: name 'df' is not defined